In [30]:
#file:/usr/hdp/2.4.0.0-169/spark/
#/usr/hdp/2.4.0.0-169/spark/data/mllib/

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import numpy as np
from pandas import *
#from ggplot import *

In [31]:
# 1 - LOAD DATA
training_file = "file:/usr/hdp/2.4.0.0-169/spark/final_patientyr_data_training_4.csv"
rawDataRDD = sc.textFile(training_file).cache()
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [32]:
# 2 - CREATE ROW RDD AND CONVERT TO DATAFRAME
from pyspark.sql import Row
csvRDD = rawDataRDD.map(lambda l: l.split(","))
rowRDD = csvRDD.map(lambda p: Row(
label=float(p[0]),
RXClass_Alt=int(p[1]),
RXClass_Antiinf=int(p[2]),
RXClass_Antineo=int(p[3]),
RXClass_Bio=int(p[4]),
RXClass_Cardio=int(p[5]),
RXClass_CNS=int(p[6]),
RXClass_Coag=int(p[7]),
RXClass_Gastro=int(p[8]),
RXClass_Horm=int(p[9]),
RXClass_Immo=int(p[10]),
RXClass_Metab=int(p[11]),
RXClass_Misc=int(p[12]),
RXClass_Nutri=int(p[13]),
RXClass_Psych=int(p[14]),
RXClass_Resp=int(p[15]),
RXClass_Top=int(p[16]),
RXClass_Unk=int(p[17]),
RXClass_UnkCat=int(p[18]),
PharmType_Drug=int(p[19]),
PharmType_Hosp=int(p[20]),
PharmType_Mail=int(p[21]),
PharmType_Online=int(p[22]),
PharmType_Store=int(p[23]),
PharmType_Unk=int(p[24]),
visit_count=int(p[25]),
VA=int(p[26]),
emer_stay=int(p[27]),
emer_med=int(p[28]),
oper=int(p[29]),
num_nights=int(p[30]),
emer_room=int(p[31]),
emer_rec=int(p[32]),
proc_count=int(p[33]),
Proc_11=int(p[34]),
Proc_13=int(p[35]),
Proc_14=int(p[36]),
Proc_16=int(p[37]),
Proc_18=int(p[38]),
Proc_2=int(p[39]),
Proc_20=int(p[40]),
Proc_21=int(p[41]),
Proc_22=int(p[42]),
Proc_23=int(p[43]),
Proc_24=int(p[44]),
Proc_27=int(p[45]),
Proc_28=int(p[46]),
Proc_3=int(p[47]),
Proc_31=int(p[48]),
Proc_33=int(p[49]),
Proc_35=int(p[50]),
Proc_36=int(p[51]),
Proc_37=int(p[52]),
Proc_38=int(p[53]),
Proc_39=int(p[54]),
Proc_4=int(p[55]),
Proc_40=int(p[56]),
Proc_42=int(p[57]),
Proc_44=int(p[58]),
Proc_45=int(p[59]),
Proc_46=int(p[60]),
Proc_47=int(p[61]),
Proc_48=int(p[62]),
Proc_49=int(p[63]),
Proc_50=int(p[64]),
Proc_51=int(p[65]),
Proc_53=int(p[66]),
Proc_54=int(p[67]),
Proc_55=int(p[68]),
Proc_57=int(p[69]),
Proc_58=int(p[70]),
Proc_59=int(p[71]),
Proc_6=int(p[72]),
Proc_60=int(p[73]),
Proc_62=int(p[74]),
Proc_63=int(p[75]),
Proc_64=int(p[76]),
Proc_65=int(p[77]),
Proc_66=int(p[78]),
Proc_67=int(p[79]),
Proc_68=int(p[80]),
Proc_69=int(p[81]),
Proc_70=int(p[82]),
Proc_74=int(p[83]),
Proc_75=int(p[84]),
Proc_76=int(p[85]),
Proc_77=int(p[86]),
Proc_78=int(p[87]),
Proc_8=int(p[88]),
Proc_80=int(p[89]),
Proc_81=int(p[90]),
Proc_82=int(p[91]),
Proc_83=int(p[92]),
Proc_84=int(p[93]),
Proc_85=int(p[94]),
Proc_86=int(p[95]),
Proc_87=int(p[96]),
Proc_88=int(p[97]),
Proc_89=int(p[98]),
Proc_92=int(p[99]),
Proc_93=int(p[100]),
Proc_94=int(p[101]),
Proc_96=int(p[102]),
Proc_97=int(p[103]),
Proc_98=int(p[104]),
Proc_99=int(p[105]),
Proc_ND=int(p[106]),
emer_rsn_Baby=int(p[107]),
emer_rsn_Babypre=int(p[108]),
emer_rsn_diag=int(p[109]),
emer_rsn_oth=int(p[110]),
emer_rsn_surg=int(p[111]),
emer_rsn_treat=int(p[112]),
cond_blood=int(p[113]),
cond_circ=int(p[114]),
cond_cong=int(p[115]),
cond_dig=int(p[116]),
cond_endo=int(p[117]),
cond_genit=int(p[118]),
cond_health_ctct=int(p[119]),
cond_ill_def=int(p[120]),
cond_infec=int(p[121]),
cond_poison=int(p[122]),
cond_mental=int(p[123]),
cond_musc=int(p[124]),
cond_NA=int(p[125]),
cond_neo=int(p[126]),
cond_nrvs=int(p[127]),
cond_NA2=int(p[128]),
cond_ND=int(p[129]),
cond_peri=int(p[130]),
cond_preg=int(p[131]),
cond_resp=int(p[132]),
cond_sense=int(p[133]),
cond_skin=int(p[134]),
cond_count=int(p[135])
    ))

training = sqlContext.createDataFrame(rowRDD)
training.registerTempTable("training")
#data_DF.printSchema()

In [33]:
# 3 - USE VECTOR ASSEMBLER TO CREATE FEATURES VECTOR
ignore = ['label']
assembler = VectorAssembler(
    inputCols=[x for x in training.columns if x not in ignore],
    outputCol='features')
assembler.transform(training)

DataFrame[PharmType_Drug: bigint, PharmType_Hosp: bigint, PharmType_Mail: bigint, PharmType_Online: bigint, PharmType_Store: bigint, PharmType_Unk: bigint, Proc_11: bigint, Proc_13: bigint, Proc_14: bigint, Proc_16: bigint, Proc_18: bigint, Proc_2: bigint, Proc_20: bigint, Proc_21: bigint, Proc_22: bigint, Proc_23: bigint, Proc_24: bigint, Proc_27: bigint, Proc_28: bigint, Proc_3: bigint, Proc_31: bigint, Proc_33: bigint, Proc_35: bigint, Proc_36: bigint, Proc_37: bigint, Proc_38: bigint, Proc_39: bigint, Proc_4: bigint, Proc_40: bigint, Proc_42: bigint, Proc_44: bigint, Proc_45: bigint, Proc_46: bigint, Proc_47: bigint, Proc_48: bigint, Proc_49: bigint, Proc_50: bigint, Proc_51: bigint, Proc_53: bigint, Proc_54: bigint, Proc_55: bigint, Proc_57: bigint, Proc_58: bigint, Proc_59: bigint, Proc_6: bigint, Proc_60: bigint, Proc_62: bigint, Proc_63: bigint, Proc_64: bigint, Proc_65: bigint, Proc_66: bigint, Proc_67: bigint, Proc_68: bigint, Proc_69: bigint, Proc_70: bigint, Proc_74: bigint

In [35]:
# 4 - CREATE LOGISTIC REGRESSION ESTIMATOR
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [36]:
# 5 - CREATE PIPELINE
pipeline = Pipeline(stages=[assembler, lr])

In [37]:
# 6 - CREATE PARAMETER GRID
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

In [39]:
# 7 - CROSS VALIDATION
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5) 

# 8 - Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)
#------------------------------------------------------------------------
# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(training)
selected = prediction.select("probability", "prediction")
#for row in selected.collect():
#    print(row)
selected.collect()

[Row(probability=DenseVector([0.9952, 0.0048]), prediction=0.0),
 Row(probability=DenseVector([0.9959, 0.0041]), prediction=0.0),
 Row(probability=DenseVector([0.9958, 0.0042]), prediction=0.0),
 Row(probability=DenseVector([0.9959, 0.0041]), prediction=0.0),
 Row(probability=DenseVector([0.9959, 0.0041]), prediction=0.0),
 Row(probability=DenseVector([0.9965, 0.0035]), prediction=0.0),
 Row(probability=DenseVector([0.9483, 0.0517]), prediction=0.0),
 Row(probability=DenseVector([0.9485, 0.0515]), prediction=0.0),
 Row(probability=DenseVector([0.9484, 0.0516]), prediction=0.0),
 Row(probability=DenseVector([0.9959, 0.0041]), prediction=0.0),
 Row(probability=DenseVector([0.9958, 0.0042]), prediction=0.0),
 Row(probability=DenseVector([0.9896, 0.0104]), prediction=0.0),
 Row(probability=DenseVector([0.9873, 0.0127]), prediction=0.0),
 Row(probability=DenseVector([0.9966, 0.0034]), prediction=0.0),
 Row(probability=DenseVector([0.9959, 0.0041]), prediction=0.0),
 Row(probability=DenseVec